# RDD Transformations and Actions

In [2]:
%%writefile example2.txt
first
second line
the third line
then a fourth line

Writing example2.txt


In [ ]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
sc.textFile('example2.txt')

example2.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [10]:
text_rdd = sc.textFile('example2.txt')

### Map

In [6]:
words = text_rdd.map(lambda line: line.split())

In [7]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [11]:
text_rdd.collect()

['first', 'second line', 'the third line', 'then a fourth line']

### FlatMap

In [9]:
text_rdd.flatMap(lambda line: line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

In [13]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Writing services.txt


In [14]:
services = sc.textFile('services.txt')

In [15]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

* slipt using map

In [17]:
services.map(lambda line: line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

* Remove hashtag

In [26]:
clean = services.map(lambda line: line[1:] if line[0]=='#' else line)

In [27]:
clean

PythonRDD[15] at RDD at PythonRDD.scala:48

In [30]:
clean = clean.map(lambda line: line.split())

In [31]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

## Using Key Value Pairs for Operations

Let us now begin to use methods that combine lambda expressions that use a ByKey argument. These ByKey methods will assume that your data is in a Key,Value form. 


For example let's find out the total sales per state: 

In [33]:
pairs = clean.map(lambda lst: (lst[3],lst[-1]))

In [34]:
pairs.collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

* **Assumes first column is Key**

In [38]:
rekey = pairs.reduceByKey(lambda amt1, amt2: amt1 +amt2)

In [39]:
rekey.collect()

[('State', 'Amount'),
 ('NY', '100.00750.00'),
 ('TX', '450.00200.00'),
 ('CA', '200.00500.00')]

* **Concatenating at the moment**

In [43]:
rekey = pairs.reduceByKey(lambda amt1, amt2: float(amt1) +float(amt2))

In [44]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [47]:
#Grab (state, amount)
step1 = clean.map(lambda lst: (lst[3],lst[-1]))
#Reduce by key
step2 = step1.reduceByKey(lambda amt1, amt2: float(amt1) + float(amt2))
#Get rid of header
step3 = step2.filter(lambda x: not x[0] =='State')
#sort results by amount
step4 = step3.sortBy(lambda stAmount: stAmount[1],ascending= False)
#ACTION
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

** Remember to try to use unpacking for readability. For example: **

In [48]:
x = ['ID','State','Amount']

In [51]:
def func1(lst):
    return lst[-1]

*You come back later and you will not know what -1 or 0 means*

* A better way is:

In [52]:
def func2(id_st_amt):
    #unpack values
    (Id,st,amt) = id_st_amt
    return amt

In [53]:
func1(x)

'Amount'

In [55]:
func2(x)

'Amount'

**Second is more readable**